# Covid long et offre de travail

Projet réalisé en 2025 par Lucile Aubain, Jean Lavallée et Paul Hobeika dans le cadre du cours de *Python pour la data science* enseigné en deuxième année de l'ENSAE.

<a id="introduction"></a>
# Introduction

La pandémie de Covid-19 a contaminé 800 millions de personnes et entraîné le décès de sept millions de personnes dans le monde entre 2020 et 2025, dont 150 000 en France. Ces conséquences directes du Covid-19 s'accompagnent d'effets de plus long terme sur le marché du travail.

Une littérature en médecine montre que de nombreuses personnes ayant contracté le Covid-19 et survécu développent des problèmes de santé persistants. Ces séquelles de long terme recouvrent des symptômes aussi divers que la fatigue chronique, des troubles neurologiques ou des difficultés respiratoires, qui peuvent limiter la capacité des personnes touchées à accomplir leurs activités quotidiennes, y compris leur activité professionnelle ([Ford et al., 2025](https://onlinelibrary.wiley.com/doi/abs/10.1002/ajim.70014?casa_token=NAKQSBiFLa8AAAAA%3AIUjd4EXrl4jU1-7FkTjwP6W7jjW2aly-mTo0LDxL27Oh4bVbCj11uAcFBwdYeCaNNGGWgzkhD-0v5TfP), [MacEwan et al., 2025](https://link.springer.com/article/10.1007/s11606-024-09062-5)). Ces affections sont couramment qualifiées de « Covid long » ([Groff et al., 2021](https://pubmed.ncbi.nlm.nih.gov/34643720/)).

Plusieurs acteurs publics ainsi que des médias ont avancé l’idée que ces affections de long terme auraient réduit l’offre de travail. Un rapport de la *Brookings Institution* ([2022](https://www.brookings.edu/articles/is-long-covid-worsening-the-labor-shortage/)) estime que le Covid long pourrait éloigner jusqu'à 4 millions d'Américains de l'emploi, et ainsi représenter jusqu'à 15 % des postes vacants aux Etats-Unis en 2022. Dans l'Union européenne, un rapport de la Commission européenne ([2024](https://economy-finance.ec.europa.eu/document/download/36713cbb-6cbf-4ddb-8a15-55a4f456e2cb_en?filename=eb077_en.pdf)) estime que le Covid long dans l'UE se traduit par un impact négatif sur l'offre de main-d'œuvre de 0,2 à 0,3 % en 2021 et de 0,3 à 0,5 % en 2022, soit entre 600 000 et 1 million de personnes en 2022. Ces ordres de grandeur sont similaires à ceux estimés par Abraham et Rendell ([2023](https://www.brookings.edu/wp-content/uploads/2023/03/BPEA_Spring2023_Abraham-Rendell_unembargoed.pdf)) pour les Etats-Unis, et par Espinosa Gonzalez et Suzuki ([2024](https://www.oecd.org/content/dam/oecd/en/publications/reports/2024/06/the-impacts-of-long-covid-across-oecd-countries_f662b21c/8bd08383-en.pdf)) pour l'ensemble des pays de l'OCDE. Ce canal par lequel le Covid-19 aurait affecté l'économie pourrait ainsi amener à revoir largement à la hausse son coût économique estimé, jusqu'à 1000 milliards de dollars pour l'OCDE ([Espinoza Gonzalez et Suzuki, 2024](https://www.oecd.org/content/dam/oecd/en/publications/reports/2024/06/the-impacts-of-long-covid-across-oecd-countries_f662b21c/8bd08383-en.pdf)).

Ce travail propose d'étudier l'effet d'une contamination au Covid-19 en 2020 sur l'offre de travail de long terme des salariés dans le contexte français, à partir des données de l'Enquête emploi en continu (EEC). Nous proposons d'abord une méthode pour classer les secteurs d'activités selon leur exposition à la pandémie de Covid-19. DÉVELOPPER.

Nous montrons ensuite que les salariés des secteurs exposés réduisent leur offre de travail de façon persistante, bien après le pic de la pandémie. Une analyse d’événement indique que les travailleurs des secteurs les plus exposés au Covid-19 en 2020 ont une probabilité inférieure d’environ X points de pourcentage d’être en activité un an plus tard, relativement à des travailleurs similaires des secteurs non exposés. A plus long terme, en 2024, DÉVELOPPER.

Ce travail s'inscrit en premier lieu dans la littérature qui étudie les liens entre Covid long et sortie de la population active ([Evans et al., 2021](https://www.sciencedirect.com/science/refhub/S0047-2727(23)00071-3/h0095), [Ziauddeen et al. (2022)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0264331), [Evans et al. (2021)](https://www.sciencedirect.com/science/refhub/S0047-2727(23)00071-3/h0095)). En particulier, Goda et al., ([2023](https://www.sciencedirect.com/science/article/pii/S0047272723000713)) montrent sur des données américaines que les travailleurs ayant été absents une semaine en raison du Covid-19 en 2020 ont 7 % moins de chance d'être toujours actifs un an plus tard que les travailleurs similaires qui n'ont pas manqué le travail pour raisons de santé en 2020. Cet effet de long terme de la pandémie de Covid-19 expliquerait un manque de 500 000 emplois aux Etats-Unis. A partir de données néerlandaises, Bussink et al. ([2025]()) montrent qu’une infection au Covid-19 réduit la probabilité d’emploi des nouveaux entrants sur le marché du travail de environ 1 point de pourcentage dans les trois mois qui suivent leur diplomation. L'effet plus limité du Covid long sur l'offre de travail obtenu par cette seconde étude peut s'expliquer par la plus faible exposition des jeunes adultes au Covid long par rapport à la moyenne des adultes étudiés dans l'étude de Goda et al. (2023).

Ce travail s'inscrit également dans les débats de politique économique relatifs aux travailleurs « manquants » (*missing workers*) observés dans la population active après la pandémie. Reuschke et Houston ([2022](https://www.tandfonline.com/doi/full/10.1080/13504851.2022.2098239)) montrent 80 000 personnes ont quitté leur emploi en raison du COVID long au Royaume-Uni, exacerbant ainsi la pénurie de main-d'œuvre dans le pays. Nos résultats font ainsi écho aux difficultés de recrutement rencontrées par les entreprises de certains secteurs en France entre 2021 et 2023.

Notre travail est par la suite organisé de la façon suivante. Dans la section [Installation](#installation), nous téléchargeons et installons les libraires nécessaires au bon déroulé du code ultérieur. Cette partie appelle également les fonctions utilisées par la suite. Dans la section [Données](#donnees), nous importons et nettoyons nos données avant de proposer une classification des secteurs selon leur exposition au Covid-19 dans la section [Statistiques descriptives](#descriptives). Nous analysons enfin l'effet du Covid sur l'offre de travail par secteur sur le long terme dans la section [Covid long et réduction de l'offre de travail](#modelisation), avant de conclure dans la section [Conclusion](#conclusion).


<a id="sommaire"></a>
## Sommaire
- [Introduction](#introduction)
- [Installation](#installation)
- [Données](#donnees)
- [Statistiques descriptives](#descriptives)
- [Covid long et réduction de l'offre de travail](#modelisation)
- [Conclusion](#conclusion)
- [Bibliographie complémentaire](#bibliographie)

<a id="installation"></a>
# Installation

In [1]:
!pip install -r scripts/requirements.txt

# Modules
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np
from lets_plot import *
from palmerpenguins import load_penguins
from urllib import request
from io import BytesIO
from sklearn.cluster import KMeans

# Fonctions
from scripts import import_eec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [palmerpenguins]m [lets-plot]]


/opt/python/lib/python3.13/site-packages/palmerpenguins/penguins.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


<a id="donnees"></a>
# Données

Dans le cadre de ce travail, nous analysons les données relatives aux absences issues des fichiers détails de l'Enquête emploi en continu (EEC). Un travailleur est enregistré comme absent s'il est actuellement employé mais n'a effectué aucune heure de travail durant la semaine de référence de l'EEC. De plus, la « raison principale » de leur absence est demandée à tous les travailleurs absents. Parmi les dix motifs pouvant être invoqués par les employés figure « Congé maladie (y compris arrêt après un accident au travail) ». Nous utilisons ici cette réponse comme proxy d'absence liée à la contamination par le Covid-19.

## Données de l'EEC de 2010 à 2024

### Import

Je n'ai pas trouvé les données dans la base Melodi et l'Insee mais directement en ligne sur le site. J'ai essayé d'utiliser la fonction `pd.read_csv()` pour les télécharger directement depuis l'URL, mais les données sont souvent compressées au format .zip avec un autre fichier de documentation. J'ai rédigé une fonction `pd.read_csv_from_zip()` qui importe le plus gros fichier .csv contenu dans le dossier zippé au bout de l'URL. J'ai aussi ajouté fait une copie des données sur le cloud de mon compte SSPCloud et mis un lien "backup" qu'on peut indiquer à la fonction si le lien direct ne fonctionne pas (j'ai vu que c'était tout le temps fait comme ça dans le cours).

In [2]:
# Liens vers les fichiers CSV accessibles directement depuis le site de l'Insee.
url_2024 = "https://www.insee.fr/fr/statistiques/fichier/8632441/FD_csv_EEC_2024.zip"
url_2023 = "https://www.insee.fr/fr/statistiques/fichier/8241122/FD_csv_EEC23.zip"
url_2022 = "https://www.insee.fr/fr/statistiques/fichier/7657353/FD_csv_EEC22.zip"
url_2021 = "https://www.insee.fr/fr/statistiques/fichier/6654604/FD_csv_EEC21.zip"
url_2020 = "https://www.insee.fr/fr/statistiques/fichier/5393560/fd_eec20_csv.zip"
url_2019 = "https://www.insee.fr/fr/statistiques/fichier/4809583/fd_eec19_csv.zip"
url_2018 = "https://www.insee.fr/fr/statistiques/fichier/4191029/fd_eec18_csv.zip"

# J'ai aussi enregistré les fichiers .csv non compressés dans un dossier de mon compte SSPCloud,
# normalement on devrait pouvoir y accéder sans problème à ces adresses s'il y a un problème avec les autres:
url_2024_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2024.csv"
url_2023_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_csv_EEC23.csv"
url_2022_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2022.csv"
url_2021_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2021.csv"
url_2020_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_csv_EEC20.csv"
url_2019_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2019.csv"
url_2018_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_csv_EEC18.csv"

# Si besoin, les dictionnaires de variables des différentes enquêtes sont à ces adresses (les variables ne sont pas les mêmes d'une année sur l'autre…) :
# url_doc_2018 = "https://www.insee.fr/fr/statistiques/fichier/4191029/EEC2018_dictionnaire_2020.pdf"
# url_doc_2019 = "https://www.insee.fr/fr/statistiques/fichier/4809583/EEC2019_dictionnaire_2020.pdf"
# url_doc_2020 = "https://www.insee.fr/fr/statistiques/fichier/5393560/EEC2020_Dictionnaire_Fichier_Detail.pdf"
# url_doc_2021 = "https://www.insee.fr/fr/statistiques/fichier/6654604/EEC%202021%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail.pdf"
# url_doc_2022 = "https://www.insee.fr/fr/statistiques/fichier/7657353/EEC%202022%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail.pdf"
# url_doc_2023 = "https://www.insee.fr/fr/statistiques/fichier/8241122/EEC%202023%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail_2024_07_17.pdf"
# url_doc_2024 = "https://www.insee.fr/fr/statistiques/fichier/8632441/EEC%202024%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail_2024.pdf"



In [3]:
# Import depuis le site de l'Insee :
eec24 = import_eec.read_csv_from_zip(url_2024, backup_url = url_2024_backup, sep = ";")
eec23 = import_eec.read_csv_from_zip(url_2023, backup_url = url_2023_backup, sep = ";")
eec22 = import_eec.read_csv_from_zip(url_2022, backup_url = url_2022_backup, sep = ";")
eec21 = import_eec.read_csv_from_zip(url_2021, backup_url = url_2021_backup, sep = ";")
eec20 = import_eec.read_csv_from_zip(url_2020, backup_url = url_2020_backup, sep = ";")
eec19 = import_eec.read_csv_from_zip(url_2019, backup_url = url_2019_backup, sep = ";")
eec18 = import_eec.read_csv_from_zip(url_2018, backup_url = url_2018_backup, sep = ";")

Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/8632441/FD_csv_EEC_2024.zip
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/8241122/FD_csv_EEC23.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_csv_EEC23.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/7657353/FD_csv_EEC22.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_EEC_2022.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/6654604/FD_csv_EEC21.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_EEC_2021.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/5393560/fd_eec20_csv.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_csv_EEC20.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/4809583/fd_eec19_csv.zip
Aucun mot-clé fourni : sélection du CSV le pl

In [4]:
# Variables qu'on voudrait garder (dictionnaire des codes 2018)
# - ACTEU : activité au sens du BIT (1: actifs occupés 2: chômeurs 3: inactifs)
# - AGE5 : je crois que c'est le plus détaillé au niveau de l'âge : il y a 5 classes d'âge - devient AGE6 en 2021
# - ANCENTR4 : ancienneté dans l'entreprise ou dans la fonction publique, 4 postes
# - ANNEE : année de référence de l'enquête
# - CATAU2010R : catégorie de la commune (pas sûr de comprendre les modalités)
# - CHPUB : nature de l'employeur principal
# - CSA : catégorie socioprofessionnelle pour les inactifs (2 chiffres)
# - CSE : catégorie socioprofessionnelle pour les actifs (2 chiffres)
# - CSP : catégorie socioprofessionnelle pour les actifs occupés (2 chiffres)
# - DIP11 : diplôme le plus élevé obtenu
# - DUHAB : type de temps de travail
# - EMPNBH : nombre d'heures effectuées dans l'emploi principal pendant la semaine de référence
# - EXTRIAN : pondération
# - FONCTC : fonction de la profession
# - HHC6 : nombre d'heures travaillées en moyenne dans l'emploi principal
# - JOURTR : nombre de jours travaillés en général par semaine
# - NAFANT : nature de l'activité de l'emploi antérieur (pour chômeurs et inactifs)
# - NAFG021UN : nature de l'activité de l'emploi principale (NAF2 à 21 postes)
# - NONDIC : raison de la non disponibilité pour travailler sous deux semaines
# - PASTRA : n'a pas travaillé la semaine de référence mais a un emploi rémunéré
# - PUB3FP : statut public ou privé de l'employeur principal
# - QPRC : position professionnelle dans l'emploi principal
# - RABS : raison de l'absence au travail la semaine de référence, pour les personnes qui ont un emploi rémunéré (2 : congé maladie ou accident du travail)
# - RAISNREC : raison principale de la non recherche d'emploi durant les quatre semaines s'achevant par la semaine de référence (4 : problèmes de santé)
# - RAISNSOU : raison pour laquelle l'individu ne souhaite pas travaille actuellement (3 : a des problèmes de santé)
# - RAISON : raison de la non-disponibilité pour travailler davantage (pour ceux qui souhaitent faire plus d'heures de travail) (4 : pour raisons de santé)
# - SEXE
# - SOUA : souhait d'avoir un emploi en remplacement de l'emploi actuel ou à venir
# - STAT2 : salarié (2) ou non salarié (1)
# - TPPRED : temps de travail dans l'emploi principal, redressé et imputé de la non-réponse partielle
# - TRAREF : exercice d'au moins une heure de travail durant la semaine de référence
# - TRIM : trimestre de référence de l'enquête (1 à 4)

vars = ['ACTEU', 
        'AGE5', 
        'ANCENTR4', 
        'ANNEE', 
        'CATAU2010R', 
        'CHPUB', 
        'CSA', 
        'CSE', 
        'CSP', 
        'DIP11', 
        'DUHAB', 
        'EMPNBH', 
        'EXTRIAN', 
        'FONCTC', 
        'HHC6', 
        'JOURTR', 
        'NAFANT', 
        'NAFG021UN',
        'NONDIC', 
        'PASTRA', 
        'PUB3FP', 
        'QPRC', 
        'RABS', 
        'RAISNREC', 
        'RAISNSOU', 
        'RAISON',
        'SEXE',
        'SOUA',
        'STAT2',
        'TPPRED',
        'TRAREF',
        'TRIM']

# Ça marche pour 2018, 2019 et 2020. Ensuite beaucoup de variables changent de nom (ou plus)

### Les absences pour raison de santé comme proxy de la contamination par le Covid-19

Les taux d’absence pour raisons de santé suivent très étroitement la hausse et à la baisse des taux d'hospitalisation liés au Covid-19 dans les séries temporelles sur l'année 2020. La Figure 1 présente le nombre total d'arrêt de travail pour cause de maladie et le nombre total d'hospitalisations liées au Covid-19 en France pour chaque semaine entre janvier 2020 et décembre 2020. Ces éléments laissent à penser que l'utilisation de la catégorie "arrêts de travail pour cause de maladie" constitue un bon proxy de contamination par le Covid-19 en 2020.

### Assemblage des différents millésimes

Les variables présentes dans les différentes tables ne sont pas les mêmes d'année en année. Par exemple, la variable d'âge est en 5 modalités et s'appelle `AGE5` en 2018 et 2019 puis elle passe à 6 modalités avec un découpage différent à partir de 2020 et s'appelle alors `AGE6`. Dans un premier temps, on peut ne conserver que les variables qui sont présentes dans toutes les tables, en espérant que les modalités restent les mêmes la plupart du temps. Si on veut en utiliser d'autres, il faudra ajouter des recodages préalables.

In [5]:
# On crée une liste de tous tes DataFrames
dfs = [eec18, eec19, eec20, eec21, eec22, eec23, eec24]

# On calcule l'intersection des colonnes
vars_communes = set(dfs[0].columns)
for df in dfs[1:]:
    vars_communes = vars_communes.intersection(df.columns)

# On convertit en liste triée pour lecture
vars_communes = sorted(vars_communes)
print(f"{len(vars_communes)} variables communes :")
print(vars_communes)

33 variables communes :
['AAC', 'ACTEU', 'ANCCHOM', 'ANNEE', 'CHPUB', 'DISPPLC', 'ENFRED', 'EXTRIAN', 'HALOR', 'METRODOM', 'NAFANTG004N', 'NAFG004UN', 'NAFG010UN', 'NAFG017UN', 'NAFG021UN', 'NAFG038UN', 'NAFG088UN', 'OFFICC', 'PASTRA', 'PASTRB', 'PUB3FP', 'RABS', 'RAISTP', 'SEXE', 'SOUSEMPL', 'STATUT', 'STC', 'STPLC', 'TEMP', 'TPPRED', 'TRAREF', 'TRIM', 'TXTPPRED']


Il y a donc 33 variables qui sont présentes avec le même nom dans toutes les bases de 2018 à 2024. Pour ne pas s'encombrer avec des variables qu'on utilise pas, on peut faire une liste des variables qu'on veut utiliser parmi celles-ci, qu'on peut allonger selon les besoins.

In [6]:
# Liste des variables qu'on veut utiliser
core_vars = ['ACTEU', 'RABS', 'SEXE', 'ANNEE', 'EXTRIAN']
# Concaténation des différents fichiers dans une seule tablea "eec_all"
eec_all = pd.concat(
    [eec18[core_vars], eec19[core_vars], eec20[core_vars], eec21[core_vars], eec22[core_vars], eec23[core_vars]],
    ignore_index = True)


In [7]:
# En fait on ne garde que les actifs en emploi (`ACTEU` == 1)
eec_actifs = eec_all[eec_all['ACTEU'] == 1]
eec_actifs['RABS'] = eec_actifs['RABS'].astype(str)

/tmp/ipykernel_4476/116353697.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eec_actifs['RABS'] = eec_actifs['RABS'].astype(str)


In [8]:
LetsPlot.setup_html()

In [ ]:
ggplot(eec_all, aes(x = 'SEXE')) + geom_bar()

<a id="descriptives"></a>
# Statistiques descriptives

Nous examinons à présent si, durant la pandémie, les taux d’absence pour raisons de santé ont davantage augmenté parmi les travailleurs dont les professions les exposent à un risque accru de contracter le Covid-19.

## Construction d'une mesure d'exposition au Covid-19 par secteur d'activité

Mettre ici le code pour le random forest

La Figure ci-dessous présente la relation entre les absences pour raisons de santé et le niveau d'exposition au Covid-19 par secteur.

Les résultats obtenus sont cohérents avec ceux de Mongey et al. ([2023](https://link.springer.com/article/10.1007/s10888-021-09487-6)), qui mobilisent les données O\*NET américaines pour classer les professions selon leur aptitude au télétravail et selon le degré de proximité physique avec d’autres personnes requis par les tâches habituelles.


## L'évolution des arrêts de travail pour raison de santé selon l'exposition au Covid-19

Quelques graphiques descriptifs

<a id="modelisation"></a>
# Covid long et réduction de l'offre de travail

Nous analysons à présent les effets des absences pour raisons de santé sur la participation à la population active à l’aide d’une analyse d’événement au niveau sectoriel. 

## Stratégie d'identification

### Doubles-différences

Nous comparons les secteurs exposés au Covid-19 aux secteurs non exposés, dans une fenêtre de quatre ans autour du pic de la pandémie, en 2020. Plus précisément, nous estimons d'abord la spécification suivante :
$$
Y_{it}=\alpha + \tau \text{Post}_{it} + \mathbf{X}_{it} \Lambda_k + \gamma_i + \delta_t + \varepsilon_{it}
$$
où $Y_{it}$ désigne le niveau d'activité du secteur $i$ à la période $t$, $\text{Post}_{i,t}=1[E_i\leq t\leq E_i+5]$ regroupe toutes les années post-2020 en un seul indicateur de traitement, $\mathbf{X}_{i,t}$ est un vecteur de variables de contrôle, les $\gamma_i$ sont des effets fixes par secteur, $\delta_t$ sont des effets fixes par période, et $\varepsilon_{it}$ est le terme d'erreur. Cette approche est semblable à un spécification *two-way fixed effects* (TWFE) statique. 

Notre hypothèse d’identification suppose que, hormis l'épisode d’absence pour raisons de santé en 2020, il n’existerait aucune différence moyenne de taux de participation entre les secteurs, une fois que l'on a contrôlé pour leurs caractéristiques initiales $\mathbf{X}_{it}$ ainsi que pour le secteur et la période (effets fixes $\gamma_i$ et $\delta_t$). Dans ce cadre, le terme $\tau$ capture donc l'effet propre de l'exposition au Covid-19 sur la participation au marché du travail des salariés du secteur $i$ après 2020 en moyenne.

### Étude d'évènement

Afin de mieux comprendre l'évolution temporelle des effets de l'exposition au Covid-19, nous estimons une spécification TWFE dynamique, ou "étude d'évènement", en remplaçant le terme $\text{Post}_{it}$ par des indicatrices pour chaque période qui suit l'année 2020. Plus précisément, nous estimons l'étude d’événement suivante :
$$
Y_{it}=\alpha+\sum_{\substack{j \in T \\ j \neq -1}}\tau_{j}L_{i,t+j}+\gamma_i+\delta_t+\varepsilon_{it}
$$
où $T_{i,t+j}=1(j>0)$ et $T\in\{-4,\cdots,+4\}$ de sorte que les coefficients $\tau_j$, pour $j \geq 0$, capturent les effets dynamiques de l'exposition au Covid-19 $j$ période après 2020. Nous fixons $j=-1$ comme période de référence, de sorte que $\tau_{-1}=0$ pour la période immédiatement antérieure à 2020.

Comme nous sommes ici en présence d'une expérience naturelle avec une seule période de traitement (l'année 2020), nous ne sommes pas soumis à la critique de Goodman-Bacon ([2021](https://www.google.com/search?client=safari&rls=en&q=Goodman-Bacon+A+(2021)+Difference-in-differences+with+variation+in+treatment+timing.+J+Econom+225(2)%3A254%E2%80%93277&ie=UTF-8&oe=UTF-8)) et il n'est pas nécessaire d'utiliser les estimateurs robustes de de Chaisemartin et d'Haultfoeuille ([2022](https://www.nber.org/papers/w29873)), Borusyak et al. ([2024](https://academic.oup.com/restud/article/91/6/3253/7601390)) ou encore Callaway et Sant'Anna ([2021](https://www.google.com/search?client=safari&rls=en&q=Callaway+B%2C+Sant%E2%80%99Anna+PH+(2021)+Difference-in-differences+with+multiple+time+periods.+J+Econom+225(2)%3A200%E2%80%93230&ie=UTF-8&oe=UTF-8)). Nous estimons donc cette spécification avec la méthode des moindres carrés ordinaires.

## Résultats

Les figures ci-dessous reportent les coefficients $\tau$ et $\tau_j$ expliqués ci-dessus.

<a id="conclusion"></a>
# Conclusion


<a id="bibliographie"></a>
# Bibliographie complémentaire

- Jorge A Morales-Camino, SPL07 LONG COVID IMPACT ON WORKERS AND ITS MONITORING IN OCCUPATIONAL HEALTH, *Occupational Medicine*, Volume 74, Issue Supplement_1, July 2024, Page 0, https://doi.org/10.1093/occmed/kqae023.0018 (https://academic.oup.com/occmed/article/74/Supplement_1/0/7706674)
- von Wachter, Till. 2021. "Long-Term Employment Effects from Job Losses during the COVID-19 Crisis? A Comparison to the Great Recession and Its Slow Recovery." *AEA Papers and Proceedings* 111: 481–85.
- Gottlieb M, Chen J, Yu H, et al. Work Impairment and Financial Outcomes Among Adults With vs Without Long COVID. JAMA Netw Open. 2025;8(8):e2526310. doi:10.1001/jamanetworkopen.2025.26310 montre que "financial and work outcomes were worse in participants with current LC up to 3 years after initial infection. COVID-19 vaccination was associated with better work and financial outcomes."
- https://www.sciencedirect.com/science/article/pii/S2772707624001644 montre que "Long COVID reduces work productivity" au Japon

